In [13]:
import pandas as pd
data=pd.read_csv('../raw_data/Tool_Info.csv')
data['age']=2019-data['year_of_pub']+1
data['citation_per_year']=data['num_citations']/data['age']
data.loc[data.loc[:,'aligner'] == 'HISAT2', 'citation_per_year'] = 95


# Creating new columns that combine rarer types
data.loc[data.loc[:,'indexing'] == 'hashing', 'index'] = 'Hashing'
data.loc[data.loc[:,'indexing'] == 'BWT-FM', 'index'] = 'BWT-FM'
data.loc[data.loc[:,'indexing'] == 'BWT-FM + hashing', 'index'] = 'BWT-FM & Hashing'
data.loc[data.loc[:,'indexing'] == 'suffix array', 'index'] = 'Other Suffix'
data.loc[data.loc[:,'indexing'] == 'BWT-ST', 'index'] = 'Other Suffix'
data.loc[data.loc[:,'indexing'] == 'suffix tree', 'index'] = 'Other Suffix'

data.loc[data.loc[:,'pairwise_alignment'] == 'SW', 'pairwise'] = 'Smith-Waterman'
data.loc[data.loc[:,'pairwise_alignment'] == 'HD', 'pairwise'] = 'Hamming Distance'
data.loc[data.loc[:,'pairwise_alignment'] == 'NW', 'pairwise'] = 'Needleman-Wunsch'
data.loc[data.loc[:,'pairwise_alignment'] == 'Non-DP Heuristic', 'pairwise'] = 'Non-DP Heuristic'
data.loc[data.loc[:,'pairwise_alignment'] == 'SW & NW', 'pairwise'] = 'Multiple Methods'
data.loc[data.loc[:,'pairwise_alignment'] == 'SW & Semi-Global', 'pairwise'] = 'Multiple Methods'
data.loc[data.loc[:,'pairwise_alignment'] == 'Rabin-Karp Algorithm', 'pairwise'] = 'Other DP'
data.loc[data.loc[:,'pairwise_alignment'] == 'Landau-Vishkin', 'pairwise'] = 'Other DP'
data.loc[data.loc[:,'pairwise_alignment'] == 'Sparse DP', 'pairwise'] = 'Other DP'
data.loc[data.loc[:,'pairwise_alignment'] == 'Banded Myers Bit Vector', 'pairwise'] = 'Other DP'
data.loc[data.loc[:,'pairwise_alignment'] == 'Semi-Global', 'pairwise'] = 'Other DP'

data.loc[data.loc[:,'fix_length_seed'] == 'Y', 'seeds_fixed'] = 'Yes'
data.loc[data.loc[:,'fix_length_seed'] == 'N', 'seeds_fixed'] = 'No'

data.loc[data.loc[:,'seed_chaining'] == 'Y', 'seeds_chained'] = 'Yes'
data.loc[data.loc[:,'seed_chaining'] == 'N', 'seeds_chained'] = 'No'

data.loc[data.loc[:,'spaced_seed'] == 'Y', 'seeds_spaced'] = 'Yes'
data.loc[data.loc[:,'spaced_seed'] == 'N', 'seeds_spaced'] = 'No'

data.head()

,ID,aligner,url,year_of_pub,application,indexing,fix_length_seed,spaced_seed,seed_chaining,pairwise_alignment,...,num_citations,wrapper,wrapper_of,age,citation_per_year,index,pairwise,seeds_fixed,seeds_chained,seeds_spaced
0,104.0,BatMeth2,https://github.com/GuoliangLi-HZAU/BatMeth2/,2019.0,BS-Seq,BWT-FM,Y,N,Y,SW & NW,...,2.0,N,NaN,1.0,2.000000,BWT-FM,Multiple Methods,Yes,Yes,No
1,63.0,BS-Seeker2,https://github.com/BSSeeker/BSseeker2,2013.0,BS-Seq,BWT-FM,Y,N,Y,SW & NW,...,173.0,Y,Bowtie2,7.0,24.714286,BWT-FM,Multiple Methods,Yes,Yes,No
2,54.0,BRAT-BW,http://compbio.cs.ucr.edu/brat/,2012.0,BS-Seq,BWT-FM,N,N,N,HD,...,70.0,N,NaN,8.0,8.750000,BWT-FM,Hamming Distance,No,No,No
3,43.0,Bismark,https://github.com/FelixKrueger/Bismark,2011.0,BS-Seq,BWT-FM,Y,N,Y,SW & NW,...,1550.0,Y,Bowtie2 or HISAT2,9.0,172.222222,BWT-FM,Multiple Methods,Yes,Yes,No
4,32.0,BS-Seeker,https://github.com/BSSeeker/BSseeker2,2010.0,BS-Seq,BWT-FM,Y,N,N,HD,...,215.0,Y,Bowtie,10.0,21.500000,BWT-FM,Hamming Distance,Yes,No,No


In [14]:
rt = pd.read_csv('../raw_data/compare.tools.runtime.hours.csv')
idx_rt = pd.read_csv('../raw_data/compare.tools.index.csv')
mem = pd.read_csv('../raw_data/compare.tools.memory.csv')

In [23]:
data_year = pd.DataFrame(data.loc[:,['year_of_pub', 'aligner', 'application', 'index', 
                        'pairwise', 'seeds_fixed', 'seeds_spaced', 'seeds_chained']])
data_year.loc[:,'aligner_lower'] = data_year.loc[:,'aligner'].str.lower()
rt.loc[:,'tools_lower'] = rt.loc[:,'Tools'].str.lower()
rtyear = pd.merge(rt, data_year, left_on='tools_lower', right_on='aligner_lower', how='inner')
rtyear.loc[:,'average_runtime'] = rtyear.iloc[:,1:11].mean(axis=1)
rtyear.loc[:,'std_dev'] = rtyear.iloc[:,1:11].std(axis=1)

# Imputing the one missing value
rtyear.loc[3,'ERR162843'] = rtyear.loc[3,'average_runtime'] 

# Only compare tools with DNA application
rtyear = rtyear.loc[rtyear.loc[:,'application'] != 'RNA-Seq'].reset_index(drop=True)

# Bowtie2 can be run in global or local, our data was generated using Needleman-Wunsch (Global)
rtyear.loc[rtyear.loc[:,'Tools'] == 'Bowtie2','pairwise'] = 'Needleman-Wunsch'
rtyear_summ = rtyear.loc[:,['aligner', 'year_of_pub', 'average_runtime', 'std_dev', 'application', 
                            'index', 'pairwise', 'seeds_fixed', 'seeds_spaced', 'seeds_chained']]
rtyear_summ.sort_values(by=['year_of_pub'], inplace=True)
for i in range(0,11):
    if rtyear.loc[i,'year_of_pub'] < 2013:
        rtyear.loc[i,'year_split'] = 'Before 2013'  
    else:
        rtyear.loc[i,'year_split'] = '2013 and later'
rtyear_melt = pd.melt(rtyear, id_vars = ['aligner', 'year_split', 'application', 'index', 'pairwise'], 
        value_vars = ['ERR009309', 'ERR013127', 'ERR013138', 'ERR045708', 'ERR050158', 'ERR162843', 'ERR181410',
                     'ERR183377', 'SRR061640', 'SRR360549'], value_name = 'runtime')
rtyear.to_csv(path_or_buf='../raw_data/runtime_comparison_with_tool_info.csv')
rtyear

,Tools,ERR009309,ERR013127,ERR013138,ERR045708,ERR050158,ERR162843,ERR181410,ERR183377,SRR061640,...,application,index,pairwise,seeds_fixed,seeds_spaced,seeds_chained,aligner_lower,average_runtime,std_dev,year_split
0,Bowtie2,2.364914,4.442126,6.125343,9.481851,4.480202,6.579199,5.183426,4.279699,2.942086,...,DNA,BWT-FM,Needleman-Wunsch,Yes,No,Yes,bowtie2,4.973138,2.040406,Before 2013
1,Bowtie,3.358317,5.825385,8.749844,12.251327,6.041617,7.729104,5.987389,4.563028,1.797393,...,DNA,BWT-FM,Hamming Distance,Yes,No,No,bowtie,6.102213,2.942134,Before 2013
2,BWA,4.048227,6.964193,8.374950,10.337639,5.155173,7.040740,6.394736,4.827655,3.171607,...,DNA,BWT-FM,Other DP,No,No,No,bwa,6.239311,2.106996,Before 2013
3,gsnap,18.782465,29.469896,40.606409,34.717148,11.450009,20.924469,16.579589,13.286443,10.444190,...,DNA,Hashing,Non-DP Heuristic,Yes,No,Yes,gsnap,20.924469,10.347573,Before 2013
4,hisat2,0.582257,2.636554,1.459196,1.739013,0.778033,1.107651,0.944994,0.820202,0.604172,...,DNA,BWT-FM,Non-DP Heuristic,Yes,No,No,hisat2,1.158713,0.634037,2013 and later
5,last,10.140177,16.316454,27.292700,27.709176,23.393876,34.322943,17.217662,14.567814,13.563960,...,DNA/BS-Seq/protein/RNA,Other Suffix,Multiple Methods,No,Yes,No,last,20.221401,7.627339,Before 2013
6,minimap2,0.955566,1.781175,3.408856,2.273320,1.625731,2.278983,1.980141,1.624609,1.086941,...,DNA/RNA-Seq,Hashing,Needleman-Wunsch,Yes,No,Yes,minimap2,1.866426,0.694104,2013 and later
7,rmap,7.669205,10.087773,14.721393,21.384636,13.965002,17.182582,12.655798,9.048157,10.189939,...,DNA,Hashing,Hamming Distance,Yes,No,No,rmap,12.950420,4.114427,Before 2013
8,smalt,12.295481,26.736237,47.018981,60.391744,29.646617,38.529345,33.415141,27.376229,16.051896,...,DNA,Hashing,Smith-Waterman,Yes,No,Yes,smalt,32.163790,14.049867,Before 2013
9,snap,1.074248,0.682443,1.840848,1.809145,0.830031,1.151981,1.163371,0.944356,0.905404,...,DNA,Hashing,Needleman-Wunsch,Yes,No,No,snap,1.165595,0.387181,Before 2013
